# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [56]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [60]:
def date_sorter():
    # Define a list to store the extracted dates
    extracted_dates = []

    # Regular expressions for different date formats
    date_patterns = [
        r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})',  # mm/dd/yy or mm/dd/yyyy
        r'(\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.,]* \d{2,4})',  # Month dd, yyyy
        r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.,]* \d{1,2}(?:st|nd|rd|th)?[-,.]* \d{2,4})',  # Month dd, yyyy
        r'(\d{1,2}/\d{4})',  # mm/yyyy
        r'(\d{4})'  # yyyy
    ]

    # Extract dates from each note
    for note in df:
        date_found = False
        for pattern in date_patterns:
            match = re.search(pattern, note)
            if match:
                date_str = match.group(0)
                extracted_dates.append(date_str)
                date_found = True
                break

        # If no date was found in the note, use '01/01/1900' as a default date
        if not date_found:
            extracted_dates.append('01/01/1900')

    # Convert the extracted date strings to datetime objects
    parsed_dates = [pd.to_datetime(date_str, errors='coerce') for date_str in extracted_dates]

    # Create a pandas Series with original row numbers
    date_series = pd.Series(parsed_dates, index=range(len(df)))

    # Sort the Series by dates and return it
    sorted_series = date_series.sort_values()

    # Create a Series with original row numbers
    result = pd.Series(index=sorted_series.index,range(len(df))

    return result

SyntaxError: positional argument follows keyword argument (2695466680.py, line 41)

In [58]:
result = date_sorter()
print(result)

474      0
153      1
129      2
225      3
171      4
      ... 
13     495
98     496
111    497
31     498
271    499
Length: 500, dtype: int64


In [53]:
import numpy as np
s_test = date_sorter()

def run_df_modified_check():
    """
    Check if df appears to be modified.
    """
    try:
        assert type(df) == pd.Series
        assert (df.index == pd.RangeIndex(start=0, stop=500, step=1)).all()
        assert (df.apply(type) == str).all()
        assert df.str.len().min() >= 6
        assert df.str[5].apply(ord).sum() == 38354
        print("Passed df modification check")
    except:
        print("Failed df modification check")

run_df_modified_check()

# check if running the code twice produces the same result
try:
    assert (date_sorter() == s_test).all()
    print("Passed repeatability check")
except:
    print("Failed repeatability check")

# check if the result has the expected index
try:
    assert type(date_sorter().index) == pd.RangeIndex
    assert (date_sorter().index == pd.RangeIndex(start=0, stop=500, step=1)).all()
    print("Passed index check")
except:
    print("Failed index check")

# check the tie-break sort for a sample of records where some have the same date
# note that this only tests a sample and does not check the entire answer
try:
    i_test = [s_test.index[s_test == v].values[0]
              for v in [318, 369, 493, 252, 314, 410, 490]]
    assert sorted(i_test) == i_test
    print("Passed secondary sort sample check")
except:
    print("Failed secondary sort sample check")

def run_v_check(s_test):
    """
    Check if the parsed dates appear to be correct and correctly sorted.
    The check works by producing some test checksums
    if you get for example a False entry in the agree column for
    index value 20 that would mean you have at least one incorrectly
    parsed or incorrectly sorted date in the **output** index
    range 20,21,...,29
    The results of the test are printed.
    Args:
    s_test: Series such as produced by date_sorter()
    Returns:
    None
    """
    try:
        v_check = pd.DataFrame({'correct':
        [6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
         11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
         11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
         11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
         17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797],
        'learner':[
        (s_test.iloc[10*i:(i+1)*10].values * np.array(range(1,11))).sum() for i in range(50)]},
        index=range(0,500,10)).assign(agree=lambda x:x['correct']==x['learner'])
        print("Values checksums:")
        print(v_check)
        assert v_check['agree'].all()
        print("Passed values check")
    except:
        print("Failed values check")
    return

run_v_check(s_test)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 7787-01-01 00:00:00 present at position 383